## 简介
- 利用以训练的mnist数据模型测试本数据集

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split                         # 数据集分割

from keras.utils import np_utils                                             # 分类化

Using TensorFlow backend.


### 导入模型

In [26]:
import pickle
with open('../Model/model_cnn_190602_9970.pkl', 'rb') as f:
    model = pickle.load(f)

model

### 导入数据集并分割
- label位于数据集第一列。

In [3]:
data_trainOrg = pd.read_csv("../Data/train.csv")
data_trainOrg.head(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data_trainOrg.shape

(42000, 785)

In [5]:
ser_label = data_trainOrg.iloc[:, 0]
df_train = data_trainOrg.iloc[:, 1:]

ser_label.head(3), df_train.shape

(0    1
 1    0
 2    1
 Name: label, dtype: int64, (42000, 784))

In [6]:
# 根据模型更改X,y
X = np.array(df_train).reshape(df_train.shape[0], 28, 28, 1).astype('float32')
y = np.array(ser_label)
y = np_utils.to_categorical(y)

X.shape, y.shape

((42000, 28, 28, 1), (42000, 10))

In [7]:
data_testOrg = pd.read_csv("../Data/test.csv")
data_testOrg.head(3)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
data_testOrg.shape

(28000, 784)

In [22]:
# 根据模型更改X
X_tar = np.array(data_testOrg).reshape(data_testOrg.shape[0], 28, 28, 1).astype('float32')

X_tar.shape

(28000, 28, 28, 1)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((29400, 28, 28, 1), (12600, 28, 28, 1), (29400, 10), (12600, 10))

### 测试模型准确率

In [27]:
model

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)              0         
__________

In [30]:
%%time
scores = model.evaluate(X, y, verbose=0)[1]*100

Wall time: 1min 6s


In [31]:
scores

99.58333333333333

## 预测测试集

In [37]:
%%time
l = []
l.append(model.predict_classes(X_test))

Wall time: 40.6 s


In [41]:
lst_result = list(l[0])

len(lst_result), lst_result[:10]

(28000, [2, 0, 9, 9, 3, 7, 0, 3, 0, 3])

In [44]:
df_result = pd.DataFrame(columns=['ImageId','Label'], index=None)

df_result['ImageId'] = [i for i in range(1, len(lst_result)+1)]
df_result['Label'] = lst_result

df_result.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [46]:
df_result.to_csv('../Result/result_0613_99583.csv',sep=',', index=None)